In [ ]:
# %reload_ext autoreload
# %autoreload 2

In [ ]:
from tklearn.kb.loader import KnowledgeLoaderConfig, AutoKnowledgeLoader

config = KnowledgeLoaderConfig.from_dict({
    "identifier": "conceptnet",
})

loader = AutoKnowledgeLoader(config)

items = set()

for i, row in enumerate(loader.iterrows()):
    items.add(row["start"]["@id"])
    items.add(row["end"]["@id"])

print(len(items))

In [ ]:
from tklearn.kb import KnowledgeBase

kb = KnowledgeBase("conceptnet/conceptnet-v5.7.0")

In [ ]:
kb.index.frozen

In [ ]:
vocab = kb.get_vocab()

In [ ]:
vocab.get_index("[UNK]")

In [ ]:
# kb.index.find('http://conceptnet.io/c/aa/a', 'http://conceptnet.io/r/ExternalURL',)

In [ ]:
from tqdm import auto as tqdm
from collections import defaultdict

from flashtext2 import KeywordProcessor


kp = KeywordProcessor(False)

label2id = defaultdict(set)

vocab = kb.get_vocab()

for id, label in tqdm.tqdm(vocab):
    label2id[label].add(id)
    kp.add_keyword(label)
    if "-" in label:
        new_label = label.replace("-", " ")
        label2id[new_label].add(id)
        kp.add_keyword(new_label)
    

In [ ]:
import base64
import random
import string

class TextEncoder:
    def __init__(self, salt_length=8):
        """Initialize the encoder with optional salt length."""
        self.salt_length = salt_length

    def _generate_salt(self):
        """Generate a random salt string."""
        return ''.join(random.choices(string.ascii_letters + string.digits, k=self.salt_length))

    def encode(self, text):
        """
        Encode text using base64 encoding with added salt.
        Returns tuple of (encoded_text, salt) for later decoding.
        """
        if not isinstance(text, str):
            raise TypeError("Input must be a string")

        # Generate a random salt
        salt = self._generate_salt()
        
        # Combine salt and text
        salted_text = salt + text
        
        # Convert to bytes and encode using base64
        text_bytes = salted_text.encode('utf-8')
        encoded_bytes = base64.b64encode(text_bytes)
        
        # Convert back to string for storage/transmission
        encoded_text = encoded_bytes.decode('utf-8')
        
        return encoded_text, salt

    def decode(self, encoded_text, salt):
        """
        Decode the encoded text using the original salt.
        """
        try:
            # Convert from base64 back to bytes
            decoded_bytes = base64.b64decode(encoded_text)
            
            # Convert bytes back to string
            decoded_text = decoded_bytes.decode('utf-8')
            
            # Remove the salt from the beginning
            if decoded_text.startswith(salt):
                return decoded_text[len(salt):]
            else:
                raise ValueError("Invalid salt or corrupted data")
            
        except base64.binascii.Error:
            raise ValueError("Invalid base64 encoding")
        except UnicodeDecodeError:
            raise ValueError("Unable to decode the text")

# Create encoder instance
encoder = TextEncoder()

# Encode the text
try:
    encoded_text, salt ="eGdWOGlvcldGb3Igc3RhcnRlcnMgYmVuZCBvdmVyIHRoZSBvbmUgaW4gcGluayBhbmQga2ljayB0aGF0IGFzcyBhbmQgcHVzc3kgdG8gZ2V0IGEgdGFzdGUgdW50aWwgc2hlJ3MgYmVnZ2luZyBmb3IgYSBkaWNrIGluc2lkZSBoZXIu", "xgV8iorW"
    
    # Decode the text
    decoded_text = encoder.decode(encoded_text, salt)
    print(f"\nDecoded text: {decoded_text}")
    
    # Verify the result
    print(f"\nSuccessful decode: {original_text == decoded_text}")
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
from rdflib import URIRef
import urllib


def uriref(val: str) -> URIRef:
    # urllib.parse.urlparse("https://conceptnet.io/r/ExternalURL")
    q = urllib.parse.urlparse(val)
    # <scheme>://<netloc>/<path>;<params>?<query>#<fragment>
    q = q._replace(
        path=urllib.parse.quote(q.path),
        params=urllib.parse.quote(q.params),
        query=urllib.parse.quote(q.query),
        fragment=urllib.parse.quote(q.fragment),
    )
    val = urllib.parse.urlunparse(q)
    # val = urllib.parse.quote(val)
    return URIRef(val)



# triger warning: explicit message from the dataset
text_ = decoded_text

triplets = {}

for token, i, j in kp.extract_keywords_with_span(text_.replace("-", " ")):
    # print(token, i, j, kb.find_neighbors(label2id[token]))
    key = (i, j, token)
    triplets[key] = []
    for id in label2id[token]:
        triplets[key] += kb.index.find(id)

In [ ]:
triplets